In [ ]:
# A
!pip install -U diffusers==0.30.3 transformers==4.45.2 accelerate==0.34.2 safetensors==0.4.5
# B
!pip install -q diffusers==0.30.3 transformers==4.45.2 accelerate==0.34.2 safetensors==0.4.5 pillow ipython
# C
!pip install -q diffusers==0.30.3 transformers==4.45.2 accelerate==0.34.2 safetensors==0.4.5 pillow
# D

# E

A — Минимальная генерация (Diffusers pipeline, text→image)


In [ ]:
import torch
from diffusers import StableDiffusionPipeline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    dtype=torch.float16,     # новая форма параметра, вместо torch_dtype
)
pipe.to(device)

In [ ]:
prompt = "a fantasy city of glass, photorealistic, sunset lighting, high detail"
image = pipe(prompt, guidance_scale=7.5, num_inference_steps=25).images[0]
image.save("/content/demo_result.png")
image

B — Демонстрация влияния параметров (петля: seed, guidance_scale, steps)


In [ ]:
import torch
from diffusers import StableDiffusionPipeline
from PIL import Image
from IPython.display import display

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# Загружаем пайплайн Stable Diffusion (можно менять модель)
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    dtype=torch.float16,
)
pipe.to(device)

In [ ]:
# === Параметрический эксперимент ===
prompts = [
  "a fantasy landscape with neon mushrooms, ultra detailed",
  "a portrait of an astronaut in a dreamy watercolor style"
]
seeds = [42, 1234, 2025]
guidances = [3.5, 7.5, 12.0]
steps = [30]  # можно варьировать steps, чтобы не тратить время

In [ ]:
results = []
for p in prompts:
  for s in seeds:
    for g in guidances:
      generator = torch.Generator(device).manual_seed(s)
      out = pipe(p, guidance_scale=g, num_inference_steps=steps[0], generator=generator)
      img = out.images[0]
      fname = f"/content/{p[:20].replace(' ','_')}_s{s}_g{g}_st{steps[0]}.png"
      img.save(fname)
      results.append((fname, p, s, g, steps[0]))

In [ ]:
# === Первые 4 изображения ===
for r in results[:4]:
  display(Image.open(r[0]).resize((256,256)))

C — Img2Img (превращение исходного изображения)


In [ ]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# === ЗАГРУЗКА ПАЙПЛАЙНА ===
pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",  # можно заменить на CompVis/stable-diffusion-v1-4
    dtype=torch.float16
).to(device)

In [ ]:
init_path = "caution.jpg"

try:
    init_image = Image.open(init_path).convert("RGB").resize((512, 512))
    print("Загружено фото из Drive ✅")
except Exception as e:
    print("Фото не найдено, создаю тестовую заглушку 🟡")
    init_image = Image.new("RGB", (512, 512), color=(180, 180, 220))
    draw = ImageDraw.Draw(init_image)
    draw.text((120, 230), "My Photo Placeholder", fill=(0, 0, 0))

display(init_image)


In [ ]:
# === ПРОМПТ И НАСТРОЙКИ ===
prompt = "make this photo look like a dramatic oil painting, Rembrandt style"
strength_values = [0.3, 0.5, 0.7, 0.9]

results = []

# === ГЕНЕРАЦИЯ ===
for s_val in strength_values:
    print(f"Генерация при strength = {s_val} ...")
    result = pipe_i2i(
        prompt=prompt,
        image=init_image,
        strength=s_val,
        guidance_scale=7.5,
        num_inference_steps=40
    )
    img = result.images[0]
    results.append((s_val, img))

In [ ]:
# === СОЗДАНИЕ КОЛЛАЖА ===
widths, heights = zip(*(img.size for _, img in results))
total_width = sum(widths)
max_height = max(heights) + 60  # добавляем место под подписи

collage = Image.new("RGB", (total_width, max_height), color=(30, 30, 30))

x_offset = 0
try:
    font = ImageFont.truetype("DejaVuSans-Bold.ttf", 24)
except:
    font = ImageFont.load_default()

for s_val, img in results:
    collage.paste(img, (x_offset, 0))
    draw = ImageDraw.Draw(collage)
    text = f"strength = {s_val}"

    # ✅ Современный способ вычислить размеры текста
    bbox = draw.textbbox((0, 0), text, font=font)
    text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]

    draw.text(
        (x_offset + (img.width - text_w)//2, img.height + 10),
        text,
        font=font,
        fill=(255, 255, 255)
    )
    x_offset += img.width

print("Complete")
display(collage)
collage.save("caution_result.png")